**project 1**

1. choose appropriate techniques for modelling text
2. implement k-nearest neighbor classifier
3. use k-NN classifier to assign classes to short texts
4. evaluate results using F1-scoring metric

In [162]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from collections import Counter
from scipy.sparse import csr_matrix
import string
import numpy as np
import scipy as sp
import nltk
from nltk.tokenize import word_tokenize
import sklearn as sk
from sklearn.feature_extraction.text import TfidfVectorizer
from collections import defaultdict

from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
#import necessary modules

print(".")

.


In [163]:
#first we want to read all the data and put into a sparse matrix

#read lines from 'train.dat' into 'traindata'
#data becomes list of str
with open("train.dat", "r") as fh:
    traindata = fh.readlines()

#cls is empty set to keep track of which lines are which class
cls = []

#preprocessing train.dat: store class in cls
#remove the class from beginning of lines for data processing
for i in range(len(traindata)):
    cls.append(traindata[i][0])
    #print(cls[i])
    traindata[i] = traindata[i][2:]

print(len(cls))

#preprocessing train lines
for i in range(len(traindata)):
    #preprocessing: convert to lowercase
    traindata[i] = traindata[i].lower()
    #remove punctuation
    traindata[i] = traindata[i].translate(str.maketrans("", "", string.punctuation))

print(traindata[0])

traindocs = [l.split() for l in traindata]
print(traindocs[0])


#read lines from 'test.dat' to 'testdata'
with open("test.dat", "r") as fh:
    testdata = fh.readlines()

#preprocessing test lines
for i in range(len(testdata)):
    #preprocessing: convert to lowercase
    testdata[i] = testdata[i].lower()
    #remove punctuation
    testdata[i] = testdata[i].translate(str.maketrans("", "", string.punctuation))

print(testdata[0])

testdocs = [l.split() for l in testdata]
print(testdocs[0])

102080
towns celebrate and hope amir can strike gold amir khans victory sparked scenes of celebration tonight in the two towns where the boxing sensation lives and trains family and friends in bolton and bury heaped praise on the

['towns', 'celebrate', 'and', 'hope', 'amir', 'can', 'strike', 'gold', 'amir', 'khans', 'victory', 'sparked', 'scenes', 'of', 'celebration', 'tonight', 'in', 'the', 'two', 'towns', 'where', 'the', 'boxing', 'sensation', 'lives', 'and', 'trains', 'family', 'and', 'friends', 'in', 'bolton', 'and', 'bury', 'heaped', 'praise', 'on', 'the']
music groups launch test case against website the music industry on monday launched a test case against kazaa one of the world 39s largest filesharing websites in a bid to stem rampant internet piracy

['music', 'groups', 'launch', 'test', 'case', 'against', 'website', 'the', 'music', 'industry', 'on', 'monday', 'launched', 'a', 'test', 'case', 'against', 'kazaa', 'one', 'of', 'the', 'world', '39s', 'largest', 'filesharing', 'w

In [164]:
#filter out words shorter than "minlen"
def filterLen(document, minlen):
    return[ [t for t in d if len(t) >= minlen] for d in document]


#filter words shorter than 4 characters long
traindocs = filterLen(traindocs, 4)
testdocs = filterLen(testdocs, 4)

print(len(traindocs[0]), traindocs[0])
print(len(testdocs[0]), testdocs[0])

for i in range(len(testdocs)):
    testdata[i] = ' '.join(testdocs[i])

print(testdata[0])


25 ['towns', 'celebrate', 'hope', 'amir', 'strike', 'gold', 'amir', 'khans', 'victory', 'sparked', 'scenes', 'celebration', 'tonight', 'towns', 'where', 'boxing', 'sensation', 'lives', 'trains', 'family', 'friends', 'bolton', 'bury', 'heaped', 'praise']
23 ['music', 'groups', 'launch', 'test', 'case', 'against', 'website', 'music', 'industry', 'monday', 'launched', 'test', 'case', 'against', 'kazaa', 'world', 'largest', 'filesharing', 'websites', 'stem', 'rampant', 'internet', 'piracy']
music groups launch test case against website music industry monday launched test case against kazaa world largest filesharing websites stem rampant internet piracy


In [165]:
#build matrix with all words in train.dat
from collections import Counter
from scipy.sparse import csr_matrix
def build_matrix(docs):
    r""" Build sparse matrix from a list of documents, 
    each of which is a list of word/terms in the document.  
    """
    nrows = len(docs)
    idx = {}
    tid = 0
    nnz = 0
    for d in docs:
        nnz += len(set(d))
        for w in d:
            if w not in idx:
                idx[w] = tid
                tid += 1
    ncols = len(idx)
        
    # set up memory
    ind = np.zeros(nnz, dtype=int)
    val = np.zeros(nnz, dtype=np.double)
    ptr = np.zeros(nrows+1, dtype=int)
    i = 0  # document ID / row counter
    n = 0  # non-zero counter
    # transfer values
    for d in docs:
        cnt = Counter(d)
        keys = list(k for k,_ in cnt.most_common())
        l = len(keys)
        for j,k in enumerate(keys):
            ind[j+n] = idx[k]
            val[j+n] = cnt[k]
        ptr[i+1] = ptr[i] + l
        n += l
        i += 1
            
    mat = csr_matrix((val, ind, ptr), shape=(nrows, ncols), dtype=np.double)
    mat.sort_indices()
    
    return mat


def csr_info(mat, name="", non_empy=False):
    r""" Print out info about this CSR matrix. If non_empy, 
    report number of non-empty rows and cols as well
    """
    if non_empy:
        print("%s [nrows %d (%d non-empty), ncols %d (%d non-empty), nnz %d]" % (
                name, mat.shape[0], 
                sum(1 if mat.indptr[i+1] > mat.indptr[i] else 0 
                for i in range(mat.shape[0])), 
                mat.shape[1], len(np.unique(mat.indices)), 
                len(mat.data)))
    else:
        print( "%s [nrows %d, ncols %d, nnz %d]" % (name, 
                mat.shape[0], mat.shape[1], len(mat.data)) )

def csr_l2normalize(mat, copy=False, **kargs):
    r""" Normalize the rows of a CSR matrix by their L-2 norm. 
    If copy is True, returns a copy of the normalized matrix.
    """
    if copy is True:
        mat = mat.copy()
    nrows = mat.shape[0]
    nnz = mat.nnz
    ind, val, ptr = mat.indices, mat.data, mat.indptr
    # normalize
    for i in range(nrows):
        rsum = 0.0    
        for j in range(ptr[i], ptr[i+1]):
            rsum += val[j]**2
        if rsum == 0.0:
            continue  # do not normalize empty rows
        rsum = 1.0/np.sqrt(rsum)
        for j in range(ptr[i], ptr[i+1]):
            val[j] *= rsum
            
    if copy is True:
        return mat

In [166]:
#building the matrix
mat = build_matrix(traindocs)
csr_info(mat)

 [nrows 102080, ncols 89963, nnz 2240750]


In [172]:
# k-nearest neighbor algorithm
def findNearestNeighbors(name, i=0, k=1):
    # first, find the document for the given name
    id = i
    # compute similarities of name's vector against all other name vectors
    csr_l2normalize(mat)
    x = mat[id,:]
    dots = x.dot(mat.T)
    dots[0,id] = -1 # invalidate self-similarity
    sims = list(zip(dots.indices, dots.data))
    sims.sort(key=lambda x: x[1], reverse=True)
    return [s[0] for s in sims[:k] if s[1] > 0 ]
    return [traindocs[s[0]] for s in sims[:k] if s[1] > 0 ]

In [175]:
with open("output.dat", "w") as fh:
    #for i in range(5):
    for i in range(len(testdata)):
        s = findNearestNeighbors(testdata[i], i, 1)
        fh.write(cls[s[0]])
        fh.write('\n') #546


KeyboardInterrupt: 